In [2]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [3]:
!which chromedriver

executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

/usr/local/bin/chromedriver


## Scrape the NASA Mars News Site to collect the latest News Title and Paragraph Text

In [8]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)

# HTML object
html = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

# Retrieve all elements that contain article information
article = soup.find('li', class_='slide')
news_title = article.find('div', class_="content_title").text
news_paragraph = article.find('div', class_="article_teaser_body").text
link = article.a['href']
news_article_link = 'https://mars.nasa.gov/' + link

print(news_title)
print(news_paragraph)
print(news_article_link)

NASA 'Optometrists' Verify Mars 2020 Rover's 20/20 Vision
Mars 2020 rover underwent an eye exam after several cameras were installed on the rover.
https://mars.nasa.gov//news/8499/nasa-optometrists-verify-mars-2020-rovers-2020-vision/


## Find the image url for the current Featured Mars Image

In [28]:
# URL of page to be scraped
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

# Navigate to page with image info
browser.click_link_by_partial_text('FULL IMAGE')
browser.is_element_present_by_text("more info", wait_time=1)
more_info = browser.find_link_by_partial_text("more info")
more_info.click()

# Scrape image info
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

image = soup.find('figure', class_='lede')
link = image.a['href']
featured_image_url = 'https://www.jpl.nasa.gov' + link
print(featured_image_url)


https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17440_hires.jpg


## Scrape the latest Mars weather tweet.

In [5]:
# URL of page to be scraped
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)

# Scrape weather report tweet 
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

mars_weather = soup.find('p', class_='tweet-text').text
print(mars_weather)


Happy Landiversary to MSL. In 7 years @MarsCuriosity has traveled 23 km (14 mi) and recorded air temps ranging from  -90ºC / -130ºF to 9ºC / 48.2ºF.  Its cold on Mars but ~9% of the Sols reported temps above freezing.pic.twitter.com/fcmnv4CmvY


## Use Pandas to scrape the table containing facts about the planet.

In [5]:
# URL of page to be scraped
url = 'https://space-facts.com/mars/'
browser.visit(url)

# Scrape planet profile table 
table = pd.read_html(url)
# print(table)

# Using pandas to convert to DataFrame
mars_facts_df = pd.DataFrame(table[1])
mars_facts_df.columns=['Description','Value']
# display(mars_facts_df)

# convert dataframe to html 
mars_facts_html = mars_facts_df.to_html()
mars_facts_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Description</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium 

## Find the title and image url for each hemisphere

In [11]:
# URL of page to be scraped
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

# Create list to hold info
hemisphere_images = []

# Scrape for info
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

results = soup.find("div", class_ = "collapsible results" )
hemispheres = results.find_all("div", class_="item")
# print(hemispheres)

# Loop through each hemisphere
for hemisphere in hemispheres:
    title = hemisphere.find("h3").text
    title = title.replace("Enhanced", "")
    src = hemisphere.find("a")["href"]
    image_link = "https://astrogeology.usgs.gov/" + src    
    browser.visit(image_link)
    html = browser.html
    soup=BeautifulSoup(html, "html.parser")
    downloads = soup.find("div", class_="downloads")
    image_url = downloads.find("a")["href"]
    hemisphere_images.append({"title": title, "img_url": image_url})
    
print(hemisphere_images)



[{'title': 'Cerberus Hemisphere ', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere ', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere ', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere ', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
